Dependências

In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp

In [2]:
# Video pose capture

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [3]:
def video_capture_detect(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)


In [4]:
cap = cv2.VideoCapture(0)
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
FPS = int(cap.get(cv2.CAP_PROP_FPS)) 

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
      
        image, results = video_capture_detect(frame, pose)
        
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        draw_landmarks(image, results)               
        
        cv2.imshow('Pose Detection', image)
        
        if cv2.waitKey(10) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

# Detecção de pose

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    resultado = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(resultado)

num_landmarks = len(landmarks)
num_values = len(resultado)
num_input_values = num_landmarks*num_values

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose

actions = np.array(['curl'])


# Extraindo frames de vídeo

In [ ]:
CAMINHO_VIDEO = os.path.join(os. getcwd(),'data') 

if not os.path.exists(CAMINHO_VIDEO):
    os.makedirs(CAMINHO_VIDEO)

actions = np.array(['curl'])
num_classes = len(actions)

no_sequences = 30

sequence_length = FPS*1

# Mudar a cada novo vídeo
start_folder = 1

for action in actions:     
    for sequence in range(start_folder,no_sequences+start_folder):
        try: 
            os.makedirs(os.path.join(CAMINHO_VIDEO, action, str(sequence)))  
        except:
            pass

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for idx, action in enumerate(actions):
        for sequence in range(start_folder, start_folder+no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = cap.read()
                
                image, results = video_capture_detect(frame, pose)

                try:
                    landmarks = results.pose_landmarks.landmark
                except:
                    pass
                
                draw_landmarks(image, results) 

                if frame_num == 0:
                    cv2.putText(image, 'CAPTURA DE COLEÇÃO', (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    
                    cv2.putText(image, 'Coletando {} Video # {}'.format(action, sequence), (15,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 8, cv2.LINE_AA)
                    cv2.putText(image, 'Coletando {} Video # {}'.format(action, sequence), (15,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, colors[idx], 4, cv2.LINE_AA)
                    
                    # Show to screen
                    cv2.imshow('Deteccao de Pose', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Coletando Video # {}'.format(action, sequence), (15,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 8, cv2.LINE_AA)
                    cv2.putText(image, 'Coletando {} Video # {}'.format(action, sequence), (15,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, colors[idx], 4, cv2.LINE_AA)
                    
                    cv2.imshow('Deteccao de Pose', image)

                keypoints = extract_keypoints(results)
                npy_path = os.path.join(CAMINHO_VIDEO, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()